In [109]:
import pandas as pd
import urllib
import requests
import json
import sys
import io
import shapely
import base64
import urllib.request
import functools
import geopandas as gp
import fiona
import numpy as np

%matplotlib inline

In [110]:
myAPI = '90341a28f02794c77f6e3abb153a4952a0bb480b'

In [111]:
keypop = 'B01003_001E'
keywpop = 'B08604_001E'
key_same_county = 'B08007_003E'

In [112]:
#read in in the variables available. the info you need is in the 1year ACS data
url = "https://api.census.gov/data/2016/acs/acs5/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

In [113]:
affkeys = np.array(list(aff1y['variables'].keys()))

In [114]:
#extracting variables of B28002 that contain "broadband"
[(k, aff1y['variables'][k]['label'])  for k in affkeys if k.startswith ("B08007") ]

[('B08007_010E', 'Estimate!!Total!!Male!!Worked outside state of residence'),
 ('B08007_011E', 'Estimate!!Total!!Female'),
 ('B08007_012E', 'Estimate!!Total!!Female!!Worked in state of residence'),
 ('B08007_013E',
  'Estimate!!Total!!Female!!Worked in state of residence!!Worked in county of residence'),
 ('B08007_014E',
  'Estimate!!Total!!Female!!Worked in state of residence!!Worked outside county of residence'),
 ('B08007_015E', 'Estimate!!Total!!Female!!Worked outside state of residence'),
 ('B08007_007E', 'Estimate!!Total!!Male!!Worked in state of residence'),
 ('B08007_008E',
  'Estimate!!Total!!Male!!Worked in state of residence!!Worked in county of residence'),
 ('B08007_009E',
  'Estimate!!Total!!Male!!Worked in state of residence!!Worked outside county of residence'),
 ('B08007_005E', 'Estimate!!Total!!Worked outside state of residence'),
 ('B08007_006E', 'Estimate!!Total!!Male'),
 ('B08007_003E',
  'Estimate!!Total!!Worked in state of residence!!Worked in county of residence

In [115]:
aff1y['variables'][key_same_county]

{'label': 'Estimate!!Total!!Worked in state of residence!!Worked in county of residence',
 'concept': 'SEX OF WORKERS BY PLACE OF WORK--STATE AND COUNTY LEVEL',
 'predicateType': 'int',
 'group': 'B08007',
 'limit': 0,
 'attributes': 'B08007_003M,B08007_003MA,B08007_003EA'}

In [116]:
year = '2013'
counties = [ 5, 47, 61, 81, 85]

url = "https://api.census.gov/data/" + year + "/acs/acs5?get=" + keywpop +\
            ",NAME&for=county:*&in=state:36&key=" + myAPI
resp = requests.request('GET', url).content
pop_work = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

pop_work = pop_work[pop_work['county'].isin(counties)]
pop_work.drop(columns=['NAME', 'state','Unnamed: 4'], inplace=True)
pop_work.rename(columns={'B08604_001E':'worker_pop'}, inplace=True)

pop_work.head(1)

,worker_pop,county
2,368562,5


In [117]:
year = '2013'

url = "https://api.census.gov/data/" + year + "/acs/acs5?get=" + keypop +\
            ",NAME&for=county:*&in=state:36&key=" + myAPI
resp = requests.request('GET', url).content
pop = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))


pop = pop[pop['county'].isin(counties)]

pop.drop(columns=['Unnamed: 4'], inplace=True)
pop.rename(columns={'B01003_001E':'resident_pop'}, inplace=True)

pop = pop.merge(pop_work, on="county")

pop.head(1)

,resident_pop,NAME,state,county,worker_pop
0,1397315,"Bronx County, New York",36,5,368562


In [118]:
year = '2013'

url = "https://api.census.gov/data/" + year + "/acs/acs5?get=" + key_same_county +\
            ",NAME&for=county:*&in=state:36&key=" + myAPI
resp = requests.request('GET', url).content
same = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))


same = same[same['county'].isin(counties)]

same.drop(columns=['NAME', 'state','Unnamed: 4'], inplace=True)
same.rename(columns={'B08007_003E':'same_res_work'}, inplace=True)

pop = pop.merge(same, on="county")

pop.head(1)

,resident_pop,NAME,state,county,worker_pop,same_res_work
0,1397315,"Bronx County, New York",36,5,368562,231832


In [122]:
pop['adjusted_day'] = pop['resident_pop'] + pop['worker_pop'] - pop['same_res_work']
pop['extra_day_people'] = pop['adjusted_day'] - pop['resident_pop'] 

In [123]:
pop

,resident_pop,NAME,state,county,worker_pop,same_res_work,adjusted_day,extra_day_people
0,1397315,"Bronx County, New York",36,5,368562,231832,1534045,136730
1,2539789,"Kings County, New York",36,47,806893,550814,2795868,256079
2,1605272,"New York County, New York",36,61,2341135,705133,3241274,1636002
3,2256400,"Queens County, New York",36,81,694443,441400,2509443,253043
4,470223,"Richmond County, New York",36,85,126249,101300,495172,24949
